In [1]:
from tools.tools import *
import pickle
from pathlib import Path
import torch
import torch.nn as nn
import optuna
torch.manual_seed(1)



In [2]:
# Load the pickled data
with open(Path("data/data.bin"), "rb") as f:
    # Load the pickled data
    data = pickle.load(f)
data = flatten_dict(data) # flatten the dictionary
data, min_max_dict = scale_dict(data) # scale the dictionary
data_lad, data_ela = format_LGHE4C25B01(min_max_dict,constant_temperature=True) # load and format the charge discharge data
datasets = {'lad': data_lad, 'ela': data_ela} # dictionary containing the above defined charge-discharge data

In [3]:
# Hyperparametertuning with Optuna
def objective(trial, training_data, testing_data):
    # Define search space
    num_layers = trial.suggest_int('num_layers', 1, 5)
    num_neurons = trial.suggest_int('num_neurons', 32, 256) # num_neurons represents both number of neurons in the FC layer as well as the number of Kernels (see CNN class implementation)
    sequence_length = trial.suggest_int("sequence_length",1,10)
    batch_size = trial.suggest_categorical("batch_size",[32,64,128,256,512,1024])
    epochs = trial.suggest_int("epochs",10,50)
    lr = trial.suggest_float("lr",1e-4,1e-2,log=True)
    
    formatted_data = []
    for i in range(0, len(training_data), 6):
        sub_dic = get_sub_dictionary(training_data, i, print_message=False)
        CNN_1D_data_format(sub_dic=sub_dic, formatted_data=formatted_data, sequence_length=sequence_length)

    # Create model with hyperparameters
    device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
    model = CNN_model(num_layers=num_layers, 
                      num_neurons=num_neurons, 
                      sequence_length=sequence_length)
    model.to(device=device)

    criterion = nn.MSELoss()  # Mean Squared Error Loss for regression tasks
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # Adam optimizer

    print(f"Trial parameters\n{trial.params}")

    # Training and validation
    training_set = CNN_1D_dataset(formatted_data)
    train_loop(model=model, 
               dataset=training_set, 
               optimizer=optimizer, 
               criterion=criterion, 
               num_epochs=epochs, 
               device=device,
               batch_size=batch_size)

    # Testing
    total_loss = 0
    for set_name, dataset in testing_data.items():
        for key, value in dataset.items():
            formatted_data = []
            CNN_1D_data_format(sub_dic=value, formatted_data=formatted_data, sequence_length=sequence_length)
            testing_set = CNN_1D_dataset(formatted_data)
            test_loss = test_model(model=model,
                                        batch_size=batch_size,
                                        test_set=testing_set,
                                        device=device,
                                        return_prediction=False)
            total_loss += test_loss
            print(f"At temperature {key} the loss is {test_loss}")
    avg_loss = total_loss / (len(datasets["ela"])+len(datasets["lad"]))
    # We want to minimize loss i.e. a smaller loss is better
    return avg_loss, model, trial.params


In [4]:
# Run objective function
n_trials = 100 # number of trials to train the cnn
study = optuna.create_study(direction='minimize') # the purpose is to minimize the loss

# Define variables to store the best model, training loss value and best parameters
best_model = None 
best_value = float("inf")
best_params = None

# If no improvement in consecutively 10 trials then drop the training
no_improvement_trials = 0
max_no_improvement_trials = 10

# Loop over the trials 
for i in range(n_trials):
    trial = study.ask()
    value, model, params = objective(trial, training_data=data, testing_data=datasets)
    study.tell(trial, value)
    if value < best_value:
        best_value = value
        best_model = model
        best_params = params
        no_improvement_trials = 0
    else:
        no_improvement_trials += 1
        print(f"Number of no improvement trials is {no_improvement_trials}")
    if no_improvement_trials >= max_no_improvement_trials:
        print(f"No improvement in the last {no_improvement_trials} trials. Stopping...")
        break

    print(f'Trial {i+1}/{n_trials} completed. Remaining trials: {n_trials - i - 1}')

best_trial = study.best_trial

# Print the found parameters
print('Best trial:')
print('  Value: ', best_trial.value)
print('  Params: ')
for key, value in best_trial.params.items():
    print('    {}: {}'.format(key, value))

[I 2023-07-10 13:17:41,620] A new study created in memory with name: no-name-fea310f9-18c5-4435-8271-53e55087a993


Trial parameters
{'num_layers': 4, 'num_neurons': 33, 'sequence_length': 4, 'batch_size': 512, 'epochs': 21, 'lr': 0.0007123911116090734}


c:\Users\Bouraoui\Documents\Job\ecm_lstm_cnn\tools\tools.py:892: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:248.)
  return torch.tensor(input, dtype=torch.float32), torch.tensor(output, dtype=torch.float32)


Epoch: 1, Train Loss: 0.004848709608275537
Epoch: 1, Validation Loss: 1.890006898002765e-05
Epoch: 2, Train Loss: 2.4802337323143655e-05
Epoch: 2, Validation Loss: 1.673956000324224e-05
Epoch: 3, Train Loss: 1.990167088553489e-05
Epoch: 3, Validation Loss: 8.496157058041522e-06
Epoch: 4, Train Loss: 1.7430953879864257e-05
Epoch: 4, Validation Loss: 1.4348688380698652e-05
Epoch: 5, Train Loss: 1.6279575289787577e-05
Epoch: 5, Validation Loss: 2.048995849231745e-05
Epoch: 6, Train Loss: 1.4155669793148719e-05
Epoch: 6, Validation Loss: 1.1855078451005806e-05
Epoch: 7, Train Loss: 1.2269480295149644e-05
Epoch: 7, Validation Loss: 8.426450009731195e-06
Epoch: 8, Train Loss: 1.099344134499679e-05
Epoch: 8, Validation Loss: 1.721067378825422e-05
Epoch: 9, Train Loss: 7.935015038718591e-06
Epoch: 9, Validation Loss: 2.2751627697275113e-06
Epoch: 10, Train Loss: 8.419543914660487e-06
Epoch: 10, Validation Loss: 3.7542938366032144e-06
Epoch: 11, Train Loss: 7.541080210519479e-06
Epoch: 11, Vali

In [5]:
# Save the best model
torch.save(best_model.state_dict(), 'best_cnn_ecm_trained.pth')